In [1]:
import configparser
import gzip
import os
import json
import requests
from json import JSONDecodeError

import pandas as pd
import joblib

from analyzer import normalize

In [2]:
def read_docs(path, files):
    meta_docs = []
    docs_text = []
    for file_name in files:
        if 'text' not in file_name:
            with gzip.open(path + file_name, 'rb') as f:
                for line in f:
                    try: 
                        meta_docs.append(json.loads(line))
                    except JSONDecodeError:
                        print("Can't read file " + filename)
        
        if 'text' in file_name:
            with gzip.open(path + file_name, 'rb') as f:
                for line in f:
                    try: 
                        docs_text.append(json.loads(line))
                    except JSONDecodeError:
                        print("Can't read file " + filename)
                    
    meta_docs = pd.DataFrame(meta_docs)
    meta_docs.drop_duplicates(subset='id', keep='first', inplace=True)
    docs_text = pd.DataFrame(docs_text)
    docs_text.drop_duplicates(subset='id_job', keep='first', inplace=True)
    docs=pd.merge(meta_docs, docs_text, how='inner', left_on='id', right_on='id_job')
    
    return docs

In [3]:
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [4]:
path = config['Data']['Path']
read_files = set()
files = set([f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))])

for file_name in files:
    if file_name not in read_files and '-01-' in file_name:
        read_files.add(file_name)
    
docs = read_docs(path, read_files)

In [5]:
vectorizer = joblib.load('../Models/vectorizer.joblib')
model = joblib.load('../Models/model.joblib')

In [6]:
docs.head()

,campaigns,checksum,company_name,company_name_hash64,date_created_x,date_expired,date_updated,emails,emails_src,html_desc_mode,...,title,title_normalized,uid,url,date_created_y,id_job,text,text_hash64,text_word_count,title_hash64
0,NaN,-3846225797890244855,,,09.01.2017 4:00:43,,08.01.2017 0:00:00,,,1,...,Работа на дому,работа на дому,-9218452320294056007,http://slanet.by/Rabota_na_domu/~8719831/~246/,09.01.2017 4:00:43,7633280191770601836,\r\n Интернет-магазину с большим ассортиментом...,921241787893745737,,-9181459132912603677
1,NaN,8615344524729648687,10-я городская поликлиника,344665408718600762,09.01.2017 16:23:43,,09.01.2017 0:00:00,,,1,...,Врач-специалист,врач специалист,-9209701696097297525,http://gsz.gov.by/ru/vacancy/316641/,09.01.2017 16:23:43,-8162932188008990604,Требования к кандидату\r\nОбразование\r\n~ВЫСШ...,5828976538653372303,,4788796866529588530
2,NaN,-1679813268377876030,,,09.01.2017 9:54:06,,08.01.2017 0:00:00,,,2,...,Дизайнер сайтов. Разработка дизайна сайта с ЛК...,дизайнер сайтов разработка дизайна сайта с лк ...,-9205884105552760265,https://www.fl.ru/projects/3090539/razrabotka-...,09.01.2017 9:54:06,-5560676161366454277,Дизайн сайта строгий деловой. То что примерно ...,-1318182559415818532,,-3992899125791179822
3,NaN,-2941287216782134415,"ЧСУП ""Легион-Нью""",-784248598256904044,09.01.2017 13:58:30,,09.01.2017 0:00:00,,,1,...,Мастер,мастер,-9203585950393933071,http://hrodna.regiony.by/работа/вакансии/#!job...,09.01.2017 13:58:30,-1118897072207195717,"Режим работы:Полный рабочий деньЗарплата:1,200...",1276204960570393937,,8335528066486724620
4,NaN,-1550848789100294706,"ООО ""Городское строительство""",7236532771465828521,09.01.2017 15:43:13,23.01.2017 0:00:00,09.01.2017 0:00:00,,,2,...,Бухгалтер,бухгалтер,-9190445575564375372,http://www.015.by/job/vacancy/full/44364,09.01.2017 15:43:13,2648512347731063678,"Бухгалтер ООО ""Городское строительство""\n\r\nу...",6399343634291149898,,8763716077634754490


In [ ]:
docs['profarea'] = ''
docs['norm_text'] = ''

for index, row in docs.iloc[:100, :].iterrows():
    if index % 5000 == 0:
        print(index)
        docs.to_csv('documents.csv', sep='\t')
    try:
        print(index)
        normalized = normalize(row['text'])
        text = [item for sublist in normalized for item in sublist]
        normalized = normalize(row['title'])
        title = [item+'_title' for sublist in normalized for item in sublist]
        
        docs.loc[index, 'norm_text'] = ' '.join(text + title)
    except:
        continue

In [ ]:
vec = vectorizer.transform(docs['norm_text'][:1000])
docs['profarea'] = model.predict(vec)[0]

In [7]:
def func(row):
    try:
        normalized = normalize(row['text'])
        text = [item for sublist in normalized for item in sublist]
        normalized = normalize(row['title'])
        title = [item+'_title' for sublist in normalized for item in sublist]
        #vec = vectorizer.transform([doc])
            #docs.loc[index, 'profarea'] = model.predict(vec)[0]
    except:
        return ''
    return ' '.join(text + title)

In [31]:
import time
start = time.time()
docs.loc[60000:, 'normalized_text'] = docs.iloc[60000:, :].apply(lambda x : func(x) ,axis=1)
print(time.time() - start)

1494.7361898422241


In [35]:
vec = vectorizer.transform(docs.normalized_text)
docs['profarea'] = model.predict(vec)

In [40]:
docs.to_csv('documents.csv', sep='\t')

In [41]:
docs = pd.read_csv('documents.csv', sep='\t')

In [42]:
docs.head()

,Unnamed: 0,campaigns,checksum,company_name,company_name_hash64,date_created_x,date_expired,date_updated,emails,emails_src,...,uid,url,date_created_y,id_job,text,text_hash64,text_word_count,title_hash64,normalized_text,profarea
0,0,NaN,-3846225797890244855,NaN,NaN,09.01.2017 4:00:43,NaN,08.01.2017 0:00:00,NaN,NaN,...,-9218452320294056007,http://slanet.by/Rabota_na_domu/~8719831/~246/,09.01.2017 4:00:43,7633280191770601836,\r\r\n Интернет-магазину с большим ассортимент...,921241787893745737,NaN,-9181459132912603677,интернет магазин больш ассортимент продукц тре...,Продажи
1,1,NaN,8615344524729648687,10-я городская поликлиника,3.446654e+17,09.01.2017 16:23:43,NaN,09.01.2017 0:00:00,NaN,NaN,...,-9209701696097297525,http://gsz.gov.by/ru/vacancy/316641/,09.01.2017 16:23:43,-8162932188008990604,Требования к кандидату\r\r\nОбразование\r\r\n~...,5828976538653372303,NaN,4788796866529588530,кандидат образован высш дополнительн педиатр у...,"Медицина, фармацевтика"
2,2,NaN,-1679813268377876030,NaN,NaN,09.01.2017 9:54:06,NaN,08.01.2017 0:00:00,NaN,NaN,...,-9205884105552760265,https://www.fl.ru/projects/3090539/razrabotka-...,09.01.2017 9:54:06,-5560676161366454277,Дизайн сайта строгий деловой. То что примерно ...,-1318182559415818532,NaN,-3992899125791179822,дизайн сайт строг делов примерн нам нужн прило...,"Искусство, развлечения, масс-медиа"
3,3,NaN,-2941287216782134415,"ЧСУП ""Легион-Нью""",-7.842486e+17,09.01.2017 13:58:30,NaN,09.01.2017 0:00:00,NaN,NaN,...,-9203585950393933071,http://hrodna.regiony.by/работа/вакансии/#!job...,09.01.2017 13:58:30,-1118897072207195717,"Режим работы:Полный рабочий деньЗарплата:1,200...",1276204960570393937,NaN,8335528066486724620,реж работ полн рабоч деньзарплат 1 200 00 руб ...,Рабочий персонал
4,4,NaN,-1550848789100294706,"ООО ""Городское строительство""",7.236533e+18,09.01.2017 15:43:13,23.01.2017 0:00:00,09.01.2017 0:00:00,NaN,NaN,...,-9190445575564375372,http://www.015.by/job/vacancy/full/44364,09.01.2017 15:43:13,2648512347731063678,"Бухгалтер ООО ""Городское строительство""\r\n\r\...",6399343634291149898,NaN,8763716077634754490,бухгалтер оо городск строительств ул весел 16 ...,"Бухгалтерия, управленческий учет, финансы пред..."
